# Poisson equation

In [ ]:
import sys, os
sys.path.append(os.path.abspath('..'))
import jax
import jax.numpy as jnp
from kfac_pinn import KFAC, pinn, pdes, training

def rhs(x):
    return (jnp.pi ** 2) * jnp.sin(jnp.pi * x)

def exact(x):
    return jnp.sin(jnp.pi * x)

key = jax.random.PRNGKey(0)
model = pinn.make_mlp(in_dim=1, key=key)
opt = KFAC(lr=1e-2)

def loss_fn(m, x):
    return pinn.interior_loss(m, x, rhs) + pinn.boundary_loss(m, jnp.array([[0.0],[1.0]]), exact)

data = [pdes.sample_interior(key, jnp.array([0.0]), jnp.array([1.0]), 64) for _ in range(200)]
model, state = training.train(model, opt, loss_fn, data, steps=200)
